### Targeted information retrieval

We have seen how to parse a webpage, which retrieves the information without distinction.

But, in general, the purpose of scraping is to automate the collection of targeted information on the web.


In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import random

Let's say I want to scrape the description of the latest movies released in theaters.

So we'll go to the Allociné website and try to find the tags that will give me links to the specific pages of these movies to get their summaries.

#### Recovery of the url of the pages of films newly released in the theaters

In [39]:
url = "http://www.allocine.fr/"
r = requests.get(url)
time.sleep(random.uniform(1.0, 2.0))
print(url, r.status_code)
soup = BeautifulSoup(r.content, "html")
soup

http://www.allocine.fr/ 200


<!DOCTYPE html>
<html lang="fr">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="index,follow,max-snippet:-1" name="robots"/>
<title>AlloCiné : Cinéma, Séries TV, BO de films et séries, Vidéos, DVD et VOD</title>
<meta content="" name="keywords"/>
<meta content="noarchive" name="Googlebot"/>
<meta content="global" name="distribution"/>
<meta content="AlloCine" name="author"/>
<meta content="France" name="country"/>
<meta content="48.87078;2.30447" name="geo.position"/>
<meta content="FR" name="geo.country"/>
<meta content="48.87078;2.30447" name="ICBM"/>
<meta content="#FECC00" name="theme-color"/>
<meta content="AlloCiné, le site de référence du cinéma et des séries tv ! Découvrez notre recherche d'horaires de films, le programme tv de vos séries préférées, l'actualité ciné et séries, les émissions

On your web browser (Chrome, Firefox,...), you can use the "inspect" function (right click -> inspect) and drag your mouse to the areas of the page that interest you. At the same time you will see the part of the HTML script highlighted that corresponds to the area that interests you.

That's how you find the tags that you are interested in.

I notice that the relative link of the web page specific to the new movie on the poster is stored in these tags:

```html
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=235582.html" title="Le Grand Bain">Le Grand Bain</a>
```


In [20]:
for p in soup.find_all("a"):
    print(p.text)

Meilleurs films d'animation Disney
Meilleurs films Pixar

            Cinéma
        

            Séries
        

            Streaming
        

            Trailers
        

            VOD
        

            Les indés
        
séances cinéma
news
dossiers
émissions AlloCiné
dernières bandes-annonces
Agenda des sorties
Kids
Meilleurs films
Meilleurs documentaires
Avant-premières
Box Office
Films pour enfants à l'affiche
DVD
Meilleurs films 2019
Tous les films pour enfants
Tous les films
Sorties de la semaine
Le Rêve américain
Marty Supreme
Super Charlie
Coutures
Maigret et le mort amoureux
L'Ourse et l'oiseau
Iron Lung
Cold Storage
Un monde fragile et merveilleux 
Le Mystérieux regard du flamant rose
Marsupilami
Gourou
Les Légendaires
Les Enfants de la Résistance
Tous les films encore à l'affiche
 Chers parents 
 The Bride! 
 Cocorico 2 
 Scream 7 
Tous les films à venir
Dernières news cinéma
The Last of Us + The Thing : ce film de science-fiction avec Liam Neeson et Joe Keery 

This time, the site is more difficult to "extract". Let's use much more specific parameters to the search function `find_all`.

In [21]:
# In addition to the tag `a`, which is easily identifiable, we notice some additional
# information such as the value of the class variable of these identical tags.
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    print(elem)

<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=1000012480.html" title="Le Rêve américain">Le Rêve américain</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=1000007317.html" title="Marty Supreme">Marty Supreme</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=326812.html" title="Super Charlie">Super Charlie</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=1000015619.html" title="Coutures">Coutures</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=325655.html" title="Maigret et le mort amoureux">Maigret et le mort amoureux</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=1000031821.html" title="L'Ourse et l'oiseau">L'Ourse et l'oiseau</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=321486.html" title="Iron Lung">Iron Lung</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=282333.html" title="Col

#### Recovery of `href`

We have noticed the presence of `href` links to the pages that interest us. Let's go get them back:

In [22]:
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    print(elem.get("href"))

/film/fichefilm_gen_cfilm=1000012480.html
/film/fichefilm_gen_cfilm=1000007317.html
/film/fichefilm_gen_cfilm=326812.html
/film/fichefilm_gen_cfilm=1000015619.html
/film/fichefilm_gen_cfilm=325655.html
/film/fichefilm_gen_cfilm=1000031821.html
/film/fichefilm_gen_cfilm=321486.html
/film/fichefilm_gen_cfilm=282333.html
/film/fichefilm_gen_cfilm=1000028071.html
/film/fichefilm_gen_cfilm=298832.html
/film/fichefilm_gen_cfilm=317669.html
/film/fichefilm_gen_cfilm=328179.html
/film/fichefilm_gen_cfilm=281028.html
/film/fichefilm_gen_cfilm=1000006174.html
/series/ficheserie_gen_cserie=1000000156.html
/series/ficheserie_gen_cserie=1000000039.html
/series/ficheserie_gen_cserie=28230.html
/series/ficheserie_gen_cserie=33867.html
/series/ficheserie_gen_cserie=37650.html
/series/ficheserie_gen_cserie=1000000881.html
/series/ficheserie_gen_cserie=1000000067.html


Can you retrieve the titles for me by searching for `"title"` in the items of the previous list?

In [45]:
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    print(elem.get("title"))

Le Rêve américain
Marty Supreme
Super Charlie
Coutures
Maigret et le mort amoureux
L'Ourse et l'oiseau
Iron Lung
Cold Storage
Un monde fragile et merveilleux 
Le Mystérieux regard du flamant rose
Marsupilami
Gourou
Les Légendaires
Les Enfants de la Résistance
Les Lionnes
Unfamiliar
A Knight of the Seven Kingdoms
Spider-Noir
Off Campus
Marshals : A Yellowstone Story
Je sais pas


#### Get summary

Let's start by building the URL that we will use to retrieve the summaries.

Start by putting the `href` values in a list of links.


In [24]:
links = []
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    links.append(elem.get("href"))
links


['/film/fichefilm_gen_cfilm=1000012480.html',
 '/film/fichefilm_gen_cfilm=1000007317.html',
 '/film/fichefilm_gen_cfilm=326812.html',
 '/film/fichefilm_gen_cfilm=1000015619.html',
 '/film/fichefilm_gen_cfilm=325655.html',
 '/film/fichefilm_gen_cfilm=1000031821.html',
 '/film/fichefilm_gen_cfilm=321486.html',
 '/film/fichefilm_gen_cfilm=282333.html',
 '/film/fichefilm_gen_cfilm=1000028071.html',
 '/film/fichefilm_gen_cfilm=298832.html',
 '/film/fichefilm_gen_cfilm=317669.html',
 '/film/fichefilm_gen_cfilm=328179.html',
 '/film/fichefilm_gen_cfilm=281028.html',
 '/film/fichefilm_gen_cfilm=1000006174.html',
 '/series/ficheserie_gen_cserie=1000000156.html',
 '/series/ficheserie_gen_cserie=1000000039.html',
 '/series/ficheserie_gen_cserie=28230.html',
 '/series/ficheserie_gen_cserie=33867.html',
 '/series/ficheserie_gen_cserie=37650.html',
 '/series/ficheserie_gen_cserie=1000000881.html',
 '/series/ficheserie_gen_cserie=1000000067.html']

The absolute URL of the movie pages we are looking for are built in this form: http://www.allocine.fr/film/fichefilm_gen_cfilm=243835.html

It is therefore necessary to take the previous list and build the absolute URLs for our search
It's up to you to play.

NB: Do not take the links for the shows(series). We only want movies.

In [25]:
movie_links = ["http://www.allocine.fr" + elem for elem in links if "film" in elem]
movie_links

['http://www.allocine.fr/film/fichefilm_gen_cfilm=1000012480.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=1000007317.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=326812.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=1000015619.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=325655.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=1000031821.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=321486.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=282333.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=1000028071.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=298832.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=317669.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=328179.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=281028.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=1000006174.html']

Finally, on each page, the title and synopsis must be retrieved. Let's try for a movie, the first of the list.

In [ ]:
url = movie_links[0]
r = requests.get(url)
time.sleep(random.uniform(1.0, 2.0))
print(url, r.status_code)
soup = BeautifulSoup(r.content, "html")

soup

http://www.allocine.fr/film/fichefilm_gen_cfilm=1000012480.html 200


<!DOCTYPE html>
<html lang="fr">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="index,follow,max-snippet:-1,max-image-preview:standard" name="robots"/>
<title>Le Rêve américain - Film 2026 - AlloCiné</title>
<meta content="" name="keywords"/>
<meta content="noarchive" name="Googlebot"/>
<meta content="global" name="distribution"/>
<meta content="AlloCine" name="author"/>
<meta content="France" name="country"/>
<meta content="48.87078;2.30447" name="geo.position"/>
<meta content="FR" name="geo.country"/>
<meta content="48.87078;2.30447" name="ICBM"/>
<meta content="#FECC00" name="theme-color"/>
<meta content="Le Rêve américain est un film réalisé par Anthony Marciano avec Jean-Pascal Zadi, Raphaël Quenard. Synopsis : Personne n'aurait parié sur Jérémy, coincé derrière le comptoir d’un vidéo club à A

For title 
```html
<div class="titlebar-title titlebar-title-lg">Le Grand Bain</div>
```
For the synopsis

```html
<section class="section ovw ovw-synopsis" id="synopsis-details">

      <div class="content-txt">
 
              C’est dans les couloirs de leur piscine municipale que Bertrand, Marcus, Simon, Laurent, Thierry et les autres s’entraînent sous l’autorité toute relative de Delphine, ancienne gloire des bassins. Ensemble, ils se sentent libres et utiles. Ils vont mettre toute leur énergie dans une discipline jusque-là propriété de la gent féminine : la natation synchronisée. Alors, oui c’est une idée plutôt bizarre, mais ce défi leur permettra de trouver un sens à leur vie...
      </div>
    
      </section>
```

In [ ]:
for elem in soup.find_all("div", attrs={"class": "titlebar-title titlebar-title-xl"}):
    # Just like that
    print(elem.text)

# Get the synopsis section
for elem in soup.find_all("section", attrs={"id": "synopsis-details"}):
    # Get the text of the synopsis
    for elem2 in elem.find_all("div", attrs={"class":"content-txt"}):
        # Just like that
        print(elem2.text)

Le Rêve américain

Personne n'aurait parié sur Jérémy, coincé derrière le comptoir d’un vidéo club à Amiens, ou sur Bouna, lorsqu'il faisait des ménages à l’aéroport d’Orly. Sans contacts, sans argent et avec un niveau d'anglais plus qu’approximatif, rien ne les prédestinait à devenir des agents qui comptent en NBA.Inspiré d’une histoire vraie, ce film raconte le parcours de deux outsiders qui, grâce à leur passion absolue pour le basket et leur amitié indéfectible, ont bravé tous les obstacles pour réaliser leur Rêve Américain.



1) Automate this script for the entire list of movies.

2) Put the information in three lists (`film_links`, `title`, `synopsis`).

3) Create a `DataFrame` object from the `pandas` library that you will have to install. This dataframe will have to include these three informations in three columns.

4) Save this dataframe in a CSV file.

And here's your first real scrap, you're real hackers now.

In [48]:
import time
import random
from random import randint

titles = []
synopsises = []

for i in range(len(movie_links)):
    url = movie_links[i]
    r = requests.get(url)
    time.sleep(random.uniform(1.0, 2.0))
    soup = BeautifulSoup(r.content, "html") 

    for elem in soup.find_all("div", attrs={"class": "titlebar-title titlebar-title-xl"}):
    # Just like that
        titles.append(elem.text)

    for elem in soup.find_all("section", attrs={"id": "synopsis-details"}):
    # Get the text of the synopsis
        for elem2 in elem.find_all("div", attrs={"class":"content-txt"}):
            # Just like that
            synopsises.append(elem2.text)


        

# Here are the things you will have to do for all links:
# - Slow down the frequency of requests to avoid being identified and therefore banned from the website.
# Use `time.sleep(random.uniform(1.0, 2.0))`
# - Get request object from URL
# - Extract the content into a variable using BeautifulSoup
# - Get title
# - Get synopsis

# Check the length of the lists before creating the dataframe
len(titles), len(synopsises), len(movie_links)

(14, 14, 14)

In [49]:
import pandas as pd

df = pd.DataFrame({"Title": titles})
df["Synopsis"] = synopsises
df["Links"] = movie_links

In [50]:
df.to_csv("./assets/allo_cine.csv", index=False)